## Problem 2: South African Insurance Management System (Implementation)

In this Script i modelled a compact but realistic insurance domain for Ubuntu Insurance (SA):
customers with risk profiles; policies made of multiple coverage lines; claims
that move through a simple workflow; and endorsements that change coverage mid-term.
I deliberately combine dataclasses (domain state & methods) with Pydantic models
(nested validation & structured inputs). I also include SA-specific constraints:
VAT=15%, bounded No-Claim Bonus, and product-type required coverages.
This shows encapsulation of pricing/claims in the dataclasses, and robust
validation/typing in the Pydantic layer.

In [10]:
# =======================[ Imports & Exceptions ]==============
from __future__ import annotations

from dataclasses import dataclass, field
from datetime import date, datetime
from enum import Enum, auto
from typing import Dict, List, Optional

from pydantic import BaseModel, Field, validator, root_validator, conint, confloat

# Domain exceptions (explicit failure modes)
class PricingError(Exception): ...
class ValidationError(Exception): ...
class PolicyStateError(Exception): ...
class ClaimStateError(Exception): ...


In [11]:
# =======================[ Enums ]=============================
class PolicyStatus(Enum):
    QUOTED = auto()
    ACTIVE = auto()
    CANCELLED = auto()
    LAPSED = auto()
    EXPIRED = auto()

class ClaimStatus(Enum):
    OPEN = auto()
    APPROVED = auto()
    DENIED = auto()
    SETTLED = auto()

class RiskBand(Enum):
    LOW = auto()
    MEDIUM = auto()
    HIGH = auto()

class ProductType(Enum):
    CAR = "CAR"
    HOME = "HOME"
    LIFE = "LIFE"
    BUSINESS = "BUSINESS"


In [12]:
# =======================[ Dataclasses (domain core) ]=========
@dataclass
class RiskProfileDC:
    prior_claims: int = 0
    age: int = 35
    location_rating: int = 3  # 1(best)..5(worst)

    def band(self) -> RiskBand:
        score = (self.location_rating - 1) + min(self.prior_claims, 3)
        if score <= 1: return RiskBand.LOW
        if score <= 3: return RiskBand.MEDIUM
        return RiskBand.HIGH

    def __str__(self) -> str:
        return f"Risk(prior_claims={self.prior_claims}, age={self.age}, loc={self.location_rating}, band={self.band().name})"

@dataclass
class CustomerDC:
    customer_id: str
    full_name: str
    risk: RiskProfileDC = field(default_factory=RiskProfileDC)

    def __repr__(self) -> str:
        return f"Customer({self.customer_id!r}, {self.full_name!r})"

@dataclass
class CoverageLineDC:
    name: str
    limit_total: float
    deductible: float
    base_rate: float
    limit_remaining: float = field(init=False)

    def __post_init__(self) -> None:
        if self.limit_total < 0 or self.deductible < 0 or self.base_rate < 0:
            raise ValidationError("CoverageLine values must be non-negative.")
        if self.deductible > self.limit_total:
            raise ValidationError("Deductible cannot exceed coverage limit.")
        self.limit_remaining = self.limit_total

    def apply_claim(self, gross_loss: float) -> float:
        eligible = max(0.0, gross_loss - self.deductible)
        payout = min(eligible, self.limit_remaining)
        self.limit_remaining = round(self.limit_remaining - payout, 2)
        return round(payout, 2)

    def __str__(self) -> str:
        return f"{self.name}(limit={self.limit_total:.2f}, remaining={self.limit_remaining:.2f}, ded={self.deductible:.2f})"

@dataclass
class EndorsementDC:
    effective_date: date
    description: str
    coverage_changes: Dict[str, Dict[str, float]] = field(default_factory=dict)
    premium_delta_annual: float = 0.0

    def apply_to(self, policy: "PolicyDC") -> None:
        for cov_name, fields in self.coverage_changes.items():
            cov = policy.coverages.get(cov_name)
            if not cov: 
                continue
            if "limit_total" in fields:
                delta = fields["limit_total"]
                cov.limit_total += delta
                cov.limit_remaining += delta
            

@dataclass(eq=True)
class PolicyDC:
    policy_id: str
    product: ProductType
    customer: CustomerDC
    start_date: date
    end_date: date
    coverages: Dict[str, CoverageLineDC] = field(default_factory=dict)
    status: PolicyStatus = PolicyStatus.QUOTED
    fees_annual: float = 150.0
    taxes_rate: float = 0.15    # SA VAT 15%
    ncb_discount: float = 0.0   # 0..0.30 typical SA no-claim bonus
    endorsements: List[EndorsementDC] = field(default_factory=list)

    def _risk_multiplier(self) -> float:
        band = self.customer.risk.band()
        return {RiskBand.LOW: 0.85, RiskBand.MEDIUM: 1.00, RiskBand.HIGH: 1.25}[band]

    def annual_base_premium(self) -> float:
        base = sum(c.base_rate for c in self.coverages.values())
        base *= self._risk_multiplier()
        base *= max(0.0, 1.0 - self.ncb_discount)
        base += sum(e.premium_delta_annual for e in self.endorsements)
        return round(base, 2)

    def annual_total_premium(self) -> float:
        taxable = self.annual_base_premium() + self.fees_annual
        total = taxable * (1 + self.taxes_rate)
        return round(total, 2)

    def bind(self) -> None:
        if self.status != PolicyStatus.QUOTED:
            raise PolicyStateError("Only QUOTED policies can be bound.")
        if self.start_date > self.end_date:
            raise ValidationError("Start date must be <= end date.")
        if not self.coverages:
            raise ValidationError("At least one coverage is required.")
        self.status = PolicyStatus.ACTIVE

    def cancel(self, reason: str = "Insured request") -> None:
        if self.status != PolicyStatus.ACTIVE:
            raise PolicyStateError("Only ACTIVE policies can be cancelled.")
        self.status = PolicyStatus.CANCELLED

    def add_endorsement(self, e: EndorsementDC) -> None:
        if not (self.start_date <= e.effective_date <= self.end_date):
            raise ValidationError("Endorsement date must be within policy term.")
        self.endorsements.append(e)
        e.apply_to(self)

    def __str__(self) -> str:
        return f"Policy({self.policy_id}, {self.product.value}, {self.status.name}, coverages={list(self.coverages)})"

@dataclass
class ClaimDC:
    claim_id: str
    policy: PolicyDC
    coverage_name: str
    loss_date: date
    reported_at: datetime
    gross_loss: float
    status: ClaimStatus = ClaimStatus.OPEN
    payout: float = 0.0
    notes: str = ""

    def assess(self) -> None:
        if self.policy.status != PolicyStatus.ACTIVE:
            raise ClaimStateError("Claims allowed only on ACTIVE policies.")
        if self.coverage_name not in self.policy.coverages:
            raise ValidationError("Coverage not on policy.")
        if not (self.policy.start_date <= self.loss_date <= self.policy.end_date):
            raise ValidationError("Loss date must be within policy term.")
        if self.gross_loss <= 0:
            raise ValidationError("Gross loss must be positive.")

    def approve(self) -> None:
        if self.status != ClaimStatus.OPEN:
            raise ClaimStateError("Only OPEN claims can be approved.")
        self.status = ClaimStatus.APPROVED

    def deny(self, reason: str) -> None:
        if self.status not in (ClaimStatus.OPEN, ClaimStatus.APPROVED):
            raise ClaimStateError("Only OPEN/APPROVED claims can be denied.")
        self.status = ClaimStatus.DENIED
        self.notes = reason
        self.payout = 0.0

    def settle(self) -> None:
        if self.status != ClaimStatus.APPROVED:
            raise ClaimStateError("Only APPROVED claims can be settled.")
        cov = self.policy.coverages[self.coverage_name]
        self.payout = cov.apply_claim(self.gross_loss)
        self.status = ClaimStatus.SETTLED

In [13]:
# =======================[ Admin services ]====================
class PolicyAdmin:
    @staticmethod
    def quote(policy: PolicyDC) -> float:
        return policy.annual_total_premium()

    @staticmethod
    def bind(policy: PolicyDC) -> None:
        policy.bind()

    @staticmethod
    def cancel(policy: PolicyDC, reason: str = "Insured request") -> None:
        policy.cancel(reason=reason)

class ClaimsManager:
    @staticmethod
    def open_claim(policy: PolicyDC, coverage_name: str, gross_loss: float, loss_date: date, claim_id: str) -> ClaimDC:
        c = ClaimDC(
            claim_id=claim_id,
            policy=policy,
            coverage_name=coverage_name,
            loss_date=loss_date,
            reported_at=datetime.utcnow(),
            gross_loss=gross_loss,
        )
        c.assess()
        return c

    @staticmethod
    def approve(claim: ClaimDC) -> None:
        claim.approve()

    @staticmethod
    def deny(claim: ClaimDC, reason: str) -> None:
        claim.deny(reason)

    @staticmethod
    def settle(claim: ClaimDC) -> None:
        claim.settle()


In [18]:
# =======================[ Pydantic Models (validation) ]======
class RiskProfileModel(BaseModel):
    prior_claims: conint(ge=0) = 0
    age: conint(ge=16, le=100) = 35
    location_rating: conint(ge=1, le=5) = 3

    def to_dc(self) -> RiskProfileDC:
        return RiskProfileDC(prior_claims=self.prior_claims, age=self.age, location_rating=self.location_rating)

class CustomerModel(BaseModel):
    customer_id: str = Field(..., min_length=3)
    full_name: str = Field(..., min_length=3)
    risk: RiskProfileModel = Field(default_factory=RiskProfileModel)

    def to_dc(self) -> CustomerDC:
        return CustomerDC(customer_id=self.customer_id, full_name=self.full_name, risk=self.risk.to_dc())

class CoverageModel(BaseModel):
    name: str
    limit_total: confloat(ge=0)  # ZAR
    deductible: confloat(ge=0)
    base_rate: confloat(ge=0)

    @validator("deductible")
    def deductible_not_exceed_limit(cls, v, values):
        limit = values.get("limit_total", 0)
        if v > limit:
            raise ValueError("Deductible cannot exceed total limit.")
        return v

    def to_dc(self) -> CoverageLineDC:
        return CoverageLineDC(
            name=self.name,
            limit_total=float(self.limit_total),
            deductible=float(self.deductible),
            base_rate=float(self.base_rate),
        )

class EndorsementModel(BaseModel):
    effective_date: date
    description: str
    coverage_changes: Dict[str, Dict[str, float]] = Field(default_factory=dict)
    premium_delta_annual: confloat(ge=0) = 0.0

    def to_dc(self) -> EndorsementDC:
        return EndorsementDC(
            effective_date=self.effective_date,
            description=self.description,
            coverage_changes=self.coverage_changes,
            premium_delta_annual=float(self.premium_delta_annual),
        )

class PolicyModel(BaseModel):
    policy_id: str = Field(..., min_length=3)
    product: ProductType
    customer: CustomerModel
    start_date: date
    end_date: date
    coverages: Dict[str, CoverageModel]
    fees_annual: confloat(ge=0) = 150.0
    taxes_rate: confloat(ge=0, le=0.20) = 0.15  # SA VAT capped at 20% here; default 15%
    ncb_discount: confloat(ge=0, le=0.30) = 0.0
    endorsements: List[EndorsementModel] = Field(default_factory=list)

    @validator("end_date")
    def end_after_start(cls, v, values):
        sd = values.get("start_date")
        if sd and v < sd:
            raise ValueError("end_date must be on/after start_date")
        return v

    @root_validator
    def product_specific_rules(cls, values):
        product = values.get("product")
        coverages: Dict[str, CoverageModel] = values.get("coverages", {})

        if not coverages:
            raise ValueError("At least one coverage is required.")

        names = set(coverages.keys())

        # --- CAR: must include Liability & Collision
        if product == ProductType.CAR:
            required = {"Liability", "Collision"}
            if not required.issubset(names):
                raise ValueError(f"CAR policies require {required} coverages.")

        # --- HOME: must include Building & Contents with minimum limits
        if product == ProductType.HOME:
            required = {"Building", "Contents"}
            if not required.issubset(names):
                raise ValueError("HOME policies require {'Building', 'Contents'}.")
            b = coverages["Building"]
            c = coverages["Contents"]
            if float(b.limit_total) < 500_000:
                raise ValueError("HOME 'Building' limit must be >= 500,000.")
            if float(c.limit_total) < 100_000:
                raise ValueError("HOME 'Contents' limit must be >= 100,000.")

        # --- LIFE: require a life benefit with zero deductible, sensible minimum sum assured
        if product == ProductType.LIFE:
            # Accept either 'Life' or 'DeathBenefit' as the primary life cover name.
            life_cov = coverages.get("Life") or coverages.get("DeathBenefit")
            if not life_cov:
                raise ValueError("LIFE policies require a 'Life' (or 'DeathBenefit') coverage.")
            if float(life_cov.deductible) != 0.0:
                raise ValueError("LIFE coverage must have deductible = 0.")
            if float(life_cov.limit_total) < 100_000:
                raise ValueError("LIFE sum assured must be >= 100,000.")

        # --- BUSINESS: liability minimum limit
        if product == ProductType.BUSINESS:
            liab = coverages.get("Liability")
            if not liab or float(liab.limit_total) < 1_000_000:
                raise ValueError("BUSINESS requires Liability limit >= 1,000,000.")

        return values

    def to_dc(self) -> PolicyDC:
        coverages_dc = {k: v.to_dc() for k, v in self.coverages.items()}
        p = PolicyDC(
            policy_id=self.policy_id,
            product=self.product,
            customer=self.customer.to_dc(),
            start_date=self.start_date,
            end_date=self.end_date,
            coverages=coverages_dc,
            fees_annual=float(self.fees_annual),
            taxes_rate=float(self.taxes_rate),
            ncb_discount=float(self.ncb_discount),
            endorsements=[e.to_dc() for e in self.endorsements],
        )
        # apply endorsements to mutate coverage if any
        for e in p.endorsements:
            e.apply_to(p)
        return p

class ClaimModel(BaseModel):
    claim_id: str = Field(..., min_length=3)
    coverage_name: str
    loss_date: date
    gross_loss: confloat(gt=0)

    def to_dc(self, policy: PolicyDC) -> ClaimDC:
        c = ClaimDC(
            claim_id=self.claim_id,
            policy=policy,
            coverage_name=self.coverage_name,
            loss_date=self.loss_date,
            reported_at=datetime.utcnow(),
            gross_loss=float(self.gross_loss),
        )
        c.assess()
        return c


In [19]:
# =======================[ Helper Printers ]===================
def print_premium_breakdown(policy: PolicyDC) -> None:
    base_sum = sum(c.base_rate for c in policy.coverages.values())
    risk_mult = policy._risk_multiplier()
    base_after_risk = base_sum * risk_mult
    base_after_ncb = base_after_risk * (1 - policy.ncb_discount)
    end_delta = sum(e.premium_delta_annual for e in policy.endorsements)
    base_with_end = base_after_ncb + end_delta
    taxable = base_with_end + policy.fees_annual
    total = taxable * (1 + policy.taxes_rate)

    print("\n— Premium Breakdown —")
    print(f"Base (sum coverages):        {base_sum:.2f}")
    print(f"× Risk multiplier:           {risk_mult:.2f}  → {base_after_risk:.2f}")
    print(f"× (1 - NCB {policy.ncb_discount:.0%}):      → {base_after_ncb:.2f}")
    print(f"+ Endorsements (annual):     {end_delta:.2f}  → {base_with_end:.2f}")
    print(f"+ Fees:                      {policy.fees_annual:.2f}  → {taxable:.2f} (taxable)")
    print(f"× (1 + VAT {policy.taxes_rate:.0%}):   → {total:.2f}  annual total")

def print_claim_result(claim: ClaimDC, cov: CoverageLineDC) -> None:
    print(f"\n— Claim {claim.claim_id} — {claim.coverage_name}")
    print(f"Gross loss: {claim.gross_loss:.2f}")
    print(f"Status:     {claim.status.name}")
    print(f"Payout:     {claim.payout:.2f}")
    print(f"Remaining limit ({cov.name}): {cov.limit_remaining:.2f}")
